In [1]:
from web3 import Web3
import json
import time
from datetime import datetime, timedelta
from web3.exceptions import ContractLogicError
import eth_abi
import random
import solcx
import os

In [2]:
arbitrum_sepolia_rpc_url = "https://sepolia-rollup.arbitrum.io/rpc"
web3 = Web3(Web3.HTTPProvider(arbitrum_sepolia_rpc_url))
assert web3.is_connected(), "Failed to connect to Arbitrum Sepolia"

In [3]:
solcx.install_solc('0.8.25')
solcx.set_solc_version('0.8.25')

def get_abi_market(file_path, contract_name):
    # Compile the contract
    with open(file_path, 'r') as file:
        contract_source_code = file.read()

    import_remappings = {
        "@openzeppelin/": "../node_modules/@openzeppelin/",
    }

    compiled_sol = solcx.compile_source(contract_source_code, output_values=['abi', 'bin'],
                                        import_remappings=import_remappings,
                                        optimize=True,
                                        via_ir=True)

    # Extract ABI and bytecode
    contract_interface = compiled_sol['<stdin>:' + contract_name]
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    return bytecode, abi

def build_and_send_tx(func, account, private_key, gas=2000000, gas_price='5', is_constructor=False, tries=1):
    for i in range(tries):
        try:
            if is_constructor:
                tx = func
            else:
                tx = func.build_transaction({
                    'from': account.address,
                    'nonce': web3.eth.get_transaction_count(account.address),
                })
            estimated_gas = int(web3.eth.estimate_gas(tx) * 2)
            base_fee = int(web3.eth.gas_price)
            max_fee_per_gas = base_fee

            tx.update({
                'gas': estimated_gas,
                'maxFeePerGas': max_fee_per_gas,
            })
            
            signed_tx = web3.eth.account.sign_transaction(tx, private_key)
            tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
            time.sleep(4)  # Wait for transaction to be mined
            return web3.eth.get_transaction_receipt(tx_hash)
        except Exception as e:
            print("Error: ", e)
            print("Will try again in 2 seconds")
            time.sleep(2)

In [4]:
(bytecode_market, market_contract_abi) = get_abi_market("../contracts/ComputationMarket.sol", "ComputationMarket")
market_contract = web3.eth.contract(bytecode=bytecode_market, abi=market_contract_abi)

(bytecode_token, comp_token_abi) = get_abi_market("../contracts/COMPToken.sol", "COMPToken")
comp_contract = web3.eth.contract(bytecode=bytecode_token, abi=comp_token_abi)

(bytecode_handlers, handlers_abi) = get_abi_market("../contracts/ComputationMarket.sol", "HandlerFunctionsCompMarket")
handler_contract = web3.eth.contract(bytecode=bytecode_handlers, abi=handlers_abi)

(bytecode_NFT, NFT_abi) = get_abi_market("../contracts/COMPNFT.sol", "CompNFT")
NFT_contract = web3.eth.contract(bytecode=bytecode_NFT, abi=NFT_abi)

deployerPrivateKey = "2ae27eeaa8095f56cd7c02adddd144bdc02d67c3d2a890b7f2ee0097cd520934"
deployer = web3.eth.account.from_key(deployerPrivateKey)
deployer_address = deployer.address

# Check deployer's balance
print("Deployers Address: ", deployer_address)
deployer_balance = web3.eth.get_balance(deployer_address)
print(f"Deployer balance: {web3.from_wei(deployer_balance, 'ether')} ETH")

tx = comp_contract.constructor(1000000 * 10 ** 18).build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})
comp_token_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("Comp token address: ", comp_token_contract_address)

tx = handler_contract.constructor().build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})
handler_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("Handler token address: ", handler_contract_address)

tx = NFT_contract.constructor().build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})
nft_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("NFT address: ", nft_contract_address)

tx = market_contract.constructor(comp_token_contract_address, nft_contract_address, handler_contract_address).build_transaction({
    'from': deployer_address,
    'nonce': web3.eth.get_transaction_count(deployer_address),
})

market_contract_address = build_and_send_tx(tx, deployer, deployerPrivateKey, is_constructor=True).contractAddress
print("Market address: ", market_contract_address)

with open('./../ComputationMarketABI.json', 'w') as file:
    file.write(json.dumps(market_contract_abi))

with open('./../COMPToken.json', 'w') as file:
    file.write(json.dumps(comp_token_abi))

with open('./../nft.json', 'w') as file:
    file.write(json.dumps(NFT_abi))

with open('./../HandlerContract.json', 'w') as file:
    file.write(json.dumps(handlers_abi))

with open('./../private_keys.json', 'r') as file:
    keys = json.load(file)

time.sleep(2)

metamask_private_keys = [
    keys["consumer"],  # Consumer
    keys["provider"],  # Provider
    keys["verifier1"],  # Verifier1
    keys["verifier2"],  # Verifier2
    keys["verifier3"],  # Verifier3
    keys["verifier4"],  # Verifier4
    keys["verifier5"]   # Verifier5
]

account_roles = [
    "Consumer",
    "Provider",
    "Verifier1",
    "Verifier2",
    "Verifier3",
    "Verifier4",
    "Verifier5"
]

market_contract = web3.eth.contract(address=market_contract_address, abi=market_contract_abi)
comp_token_contract = web3.eth.contract(address=comp_token_contract_address, abi=comp_token_abi)
nft_contract  = web3.eth.contract(address=nft_contract_address, abi=NFT_abi)

func = nft_contract.functions.transferNFTContractOwnership(market_contract_address)
receipt = build_and_send_tx(func, deployer, deployerPrivateKey)
print(f"NFT contract ownership transferred to {market_contract_address}")

def get_balance(account):
    balance = comp_token_contract.functions.balanceOf(account.address).call()
    return web3.from_wei(balance, 'ether')

def print_balance(accounts, roles):
    for account, role in zip(accounts, roles):
        print(f"Balance of {account.address} ({role}): {get_balance(account)} COMPToken")

def approve_tokens(spender_address, amount, account, private_key):
    func = comp_token_contract.functions.approve(spender_address, amount)
    receipt = build_and_send_tx(func, account, private_key)
    #print(f"{account.address} approved {amount} tokens for {spender_address}")

def transfer_tokens(to_account, amount, from_account, private_key):
    func = comp_token_contract.functions.transfer(to_account.address, amount)
    receipt = build_and_send_tx(func, from_account, private_key)
    #print(f"Transferred {amount} tokens from {from_account.address} to {to_account.address}")

def print_request(request_id):
    result = market_contract.functions.requests(request_id).call()
    descriptions = [
        "Consumer address",
        "Payment for provider",
        "Total payment for verifiers",
        "Number of operations",
        "Number of verifiers",
        "Operation file URL",
        "Computation deadline",
        "Verification deadline",
        "Total payment",
        "Completed",
        "Has been computed",
        "Number of verifiers sample size",
        "Main provider address",
        "Time allocated for verification",
        "Layer count",
        "Layer compute index",
        "Round index",
        "State",
        "Stake",
        "Payment per round for verifiers",
        "Total paid for verification",
        "Protocol version",
        "Verifier selection count",
        "First initialised time",
        "Layer size",
        "Hash of input files"
    ]

    for desc, value in zip(descriptions, result):
        print(f"{desc}: {value}")

Deployers Address:  0xA10A627707da9278f07C80983696Dc153E4714aE
Deployer balance: 0.0657771161 ETH
Comp token address:  0xA6078B4480B73698b2297c10c834C19F48cf9d9a
Handler token address:  0xaB2440A23669877ce46C71134E130522611f8B53
NFT address:  0x13a117651E363A7D91dfcFc111aD1a95E7F1aD76
Market address:  0xE00BCAD0575C18B6231e0aa30A38A75Ea250038B
NFT contract ownership transferred to 0xE00BCAD0575C18B6231e0aa30A38A75Ea250038B


In [5]:
def create_request(account, private_key):
    func = market_contract.functions.createRequest(
        100 * 10**18,
        10 * 10**18,
        6000,
        5,
        ["https://example.com/input1", "https://example.com/input2"],
        "https://example.com/operations",
        int((datetime.now() + timedelta(hours=1000000)).timestamp()),
        int((datetime.now() + timedelta(hours=2000000)).timestamp()),
        100000,
        3,
        1,
        2500,
        web3.keccak(text="example hash"),
        500 * 10**18
    )
    receipt = build_and_send_tx(func, account, private_key)
    time.sleep(5)
    request_id = market_contract.functions.requestCount().call() - 1
    print(f"Request created with ID: {request_id} by Consumer")
    print_balance([account], ["Consumer"])
    return request_id

def select_request(provider_account, private_key, request_id):
    func = market_contract.functions.selectRequest(request_id)
    receipt = build_and_send_tx(func, provider_account, private_key)
    print(f"Request {request_id} selected by Provider")
    print_balance([provider_account], ["Provider"])

def complete_request(provider_account, private_key, request_id, outputFileURLs):
    func = market_contract.functions.completeRequest(request_id, outputFileURLs)
    receipt = build_and_send_tx(func, provider_account, private_key)
    print(f"Request {request_id} completed by Provider")

def apply_for_verification(verifier_account, private_key, request_id):
    func = market_contract.functions.applyForVerificationForRequest(request_id)
    receipt = build_and_send_tx(func, verifier_account, private_key)
    print(f"Verifier {verifier_account.address} applied for verification")
    print_balance([verifier_account], ["Verifier"])

def trigger_verifier(verifier_account, private_key, request_id):
    func = market_contract.functions.chooseVerifiersForRequestTrigger(request_id)
    receipt = build_and_send_tx(func, verifier_account, private_key)
    print(f"Verifier {verifier_account.address} triggered verifier selection")

def submit_commitment(verifier_account, private_key, request_id, computed_hash):
    func = market_contract.functions.submitCommitment(request_id, computed_hash)
    receipt = build_and_send_tx(func, verifier_account, private_key)
    print(f"Verifier {verifier_account.address} submitted commitment")

def reveal_provider_key_and_hash(provider_account, private_key, request_id, answerHash, privateKeyRand, initialisationVecRand):
    func = market_contract.functions.revealProviderKeyAndHash(
        request_id,
        privateKeyRand,
        initialisationVecRand,
        answerHash 
    )
    receipt = build_and_send_tx(func, provider_account, private_key)
    print(f"Provider revealed key and hash for request {request_id}")

def reveal_commitment(verifier_account, private_key, request_id, agree, answerHash, nonce):
    func = market_contract.functions.revealCommitment(
        request_id,
        agree,
        answerHash,
        nonce 
    )
    receipt = build_and_send_tx(func, verifier_account, private_key)
    print(f"Verifier {verifier_account.address} revealed commitment")

def calculate_majority_and_reward(verifierAccount, private_key, request_id, round_num):
    func = market_contract.functions.calculateMajorityAndReward(request_id, round_num)
    receipt = build_and_send_tx(func, verifierAccount, private_key)
    print(f"Verifier {verifierAccount.address} calculated majority and reward for {request_id}, round {round_num}")
    print_balance([verifierAccount], ["Verifier"])

In [6]:
consumer_account = web3.eth.account.from_key(metamask_private_keys[0])
provider_account = web3.eth.account.from_key(metamask_private_keys[1])
verifier_accounts = [web3.eth.account.from_key(key) for key in metamask_private_keys[2:]]

all_accounts = [consumer_account, provider_account] + verifier_accounts

#reveal_commitment(verifier_accounts[0], verifier_accounts[00].key, 0, True, web3.keccak(text="hashed_answer"), web3.keccak(text="nonce_0"))

print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

# Approve tokens for consumer
approve_tokens(market_contract_address, 500 * 10**18, consumer_account, consumer_account.key)

# Transfer tokens to provider and verifiers
transfer_tokens(provider_account, 1000 * 10**18, consumer_account, consumer_account.key)
for verifier_account in verifier_accounts:
    transfer_tokens(verifier_account, 500 * 10**18, consumer_account, consumer_account.key)

print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

time.sleep(4)
# Create request

Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 1000000 COMPToken
Balance of 0xC7D6129946779d29C35CCd95c984cD1fF0633678 (Provider): 0 COMPToken
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier1): 0 COMPToken
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier2): 0 COMPToken
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier3): 0 COMPToken
Balance of 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 (Verifier4): 0 COMPToken
Balance of 0x54CD26745fD46fE30680EE504c952D0B57E377ee (Verifier5): 0 COMPToken
Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 996500 COMPToken
Balance of 0xC7D6129946779d29C35CCd95c984cD1fF0633678 (Provider): 1000 COMPToken
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier1): 500 COMPToken
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier2): 500 COMPToken
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier3): 500 COMPToken
Balance of 0x4400B62B62a2049BC24a83E

In [7]:
request_id = create_request(consumer_account, consumer_account.key)
print_request(request_id)

approve_tokens(market_contract_address, 500 * 10**18, provider_account, provider_account.key)

select_request(provider_account, provider_account.key, request_id)

outputFileURLs = ["https://example.com/output"]
complete_request(provider_account, provider_account.key, request_id, outputFileURLs)

Request created with ID: 0 by Consumer
Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 996310 COMPToken
Consumer address: 0xA10A627707da9278f07C80983696Dc153E4714aE
Payment for provider: 100000000000000000000
Total payment for verifiers: 90000000000000000000
Number of operations: 6000
Number of verifiers: 5
Operation file URL: https://example.com/operations
Computation deadline: 5323035541
Verification deadline: 8923035541
Total payment: 190000000000000000000
Completed: False
Has been computed: False
Number of verifiers sample size: 3
Main provider address: 0x0000000000000000000000000000000000000000
Time allocated for verification: 100000
Layer count: 3
Layer compute index: 0
Round index: 0
State: 0
Stake: 500000000000000000000
Payment per round for verifiers: 10000000000000000000
Total paid for verification: 0
Protocol version: 1
Verifier selection count: 0
First initialised time: 0
Layer size: 2500
Hash of input files: b"RF|Sl\x00\x83\xb7\xc5\xd0,\xe9\x8ed\xb6\xa2\x

In [8]:
print_balance([market_contract], ["market"])
print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

Balance of 0xE00BCAD0575C18B6231e0aa30A38A75Ea250038B (market): 690 COMPToken
Balance of 0xA10A627707da9278f07C80983696Dc153E4714aE (Consumer): 996310 COMPToken
Balance of 0xC7D6129946779d29C35CCd95c984cD1fF0633678 (Provider): 500 COMPToken
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier1): 500 COMPToken
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier2): 500 COMPToken
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier3): 500 COMPToken
Balance of 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 (Verifier4): 500 COMPToken
Balance of 0x54CD26745fD46fE30680EE504c952D0B57E377ee (Verifier5): 500 COMPToken


In [11]:
def isVerifierChosenForRound(verifier_account, request_id, round_number):
    return market_contract.functions.isVerifierChosenForRound(request_id, round_number + 1, verifier_account.address).call(
        {
            'from': verifier_account.address
        })

In [10]:
round_number = 0
# Apply for verification as verifiers
for verifier_account in verifier_accounts:
    approve_tokens(market_contract_address, 10 * 10**18, verifier_account, verifier_account.key)
    apply_for_verification(verifier_account, verifier_account.key, request_id)
time.sleep(6)

for verifier_account in verifier_accounts:
    trigger_verifier(verifier_account, verifier_account.key, request_id)
time.sleep(6)

print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

print(f"Starting round {round_number + 1}")

for verifier_account in verifier_accounts:
    # Example data (you can replace these with actual values)
    answer_hash = Web3.keccak(text="hashed_answer")
    nonce = Web3.keccak(text=f"nonce_{verifier_account.address}")

    verifierChosen = isVerifierChosenForRound(verifier_account, request_id, round_number)

    if verifierChosen:
        # Concatenate the byte representations of the inputs
        concatenated = Web3.solidity_keccak(
            ['bytes32', 'bytes32', 'address'],
            [answer_hash, nonce, verifier_account.address]
        )
        submit_commitment(verifier_account, verifier_account.key, request_id, concatenated)

provider_answer_hash = Web3.keccak(text="incorrect answer")
provider_key = random.randint(0, 10000)
initialisationVector = random.randint(0, 10000)
reveal_provider_key_and_hash(provider_account, provider_account.key, request_id, provider_answer_hash, provider_key, initialisationVector)

for verifier_account in verifier_accounts:
    answer_hash = Web3.keccak(text="hashed_answer")
    nonce = web3.keccak(text=f"nonce_{verifier_account.address}")

    verifierChosen = isVerifierChosenForRound(verifier_account, request_id, round_number)

    if verifierChosen:
        reveal_commitment(verifier_account, verifier_account.key, request_id, False, answer_hash, nonce)

for verifier_account in verifier_accounts:
    if isVerifierChosenForRound(verifier_account, request_id, round_number):
        calculate_majority_and_reward(verifier_account, verifier_account.key, request_id, round_number + 1)

print_request(request_id)

print(f"Balances after round {round_number + 1}:")
print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

Verifier 0x5F373754819cCA00230eCFBE55419d76329b585A applied for verification
Balance of 0x5F373754819cCA00230eCFBE55419d76329b585A (Verifier): 490 COMPToken
Verifier 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 applied for verification
Balance of 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 (Verifier): 490 COMPToken
Verifier 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 applied for verification
Balance of 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 (Verifier): 490 COMPToken
Verifier 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 applied for verification
Balance of 0x4400B62B62a2049BC24a83E77Cd9F2f4A6B72171 (Verifier): 490 COMPToken
Verifier 0x54CD26745fD46fE30680EE504c952D0B57E377ee applied for verification
Balance of 0x54CD26745fD46fE30680EE504c952D0B57E377ee (Verifier): 490 COMPToken
Verifier 0x5F373754819cCA00230eCFBE55419d76329b585A triggered verifier selection
Verifier 0x93764B46e418b16Ae60cd115CFa7b7Ab1C59e9F6 triggered verifier selection
Verifier 0xBB8a6A672e4A8A2280D734E5E19225c4beFFC561 t

ABIFunctionNotFound: ("The function 'getVerifiersChosenForRound' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')

In [ ]:
print("Done!")
print("Final Request State:")
print_request(request_id)

In [12]:
for verifier_account in verifier_accounts:
    # Example data (you can replace these with actual values)
    answer_hash = Web3.keccak(text="hashed_answer")
    nonce = Web3.keccak(text=f"nonce_{verifier_account.address}")

    verifierChosen = isVerifierChosenForRound(verifier_account, request_id, round_number)

    if verifierChosen:
        # Concatenate the byte representations of the inputs
        concatenated = Web3.solidity_keccak(
            ['bytes32', 'bytes32', 'address'],
            [answer_hash, nonce, verifier_account.address]
        )
        submit_commitment(verifier_account, verifier_account.key, request_id, concatenated)

provider_answer_hash = Web3.keccak(text="incorrect answer")
provider_key = random.randint(0, 10000)
initialisationVector = random.randint(0, 10000)
reveal_provider_key_and_hash(provider_account, provider_account.key, request_id, provider_answer_hash, provider_key, initialisationVector)

for verifier_account in verifier_accounts:
    answer_hash = Web3.keccak(text="hashed_answer")
    nonce = web3.keccak(text=f"nonce_{verifier_account.address}")

    verifierChosen = isVerifierChosenForRound(verifier_account, request_id, round_number)

    if verifierChosen:
        reveal_commitment(verifier_account, verifier_account.key, request_id, False, answer_hash, nonce)

for verifier_account in verifier_accounts:
    if isVerifierChosenForRound(verifier_account, request_id, round_number):
        calculate_majority_and_reward(verifier_account, verifier_account.key, request_id, round_number + 1)

print_request(request_id)

print(f"Balances after round {round_number + 1}:")
print_balance(all_accounts, ["Consumer", "Provider", "Verifier1", "Verifier2", "Verifier3", "Verifier4", "Verifier5"])

ABIFunctionNotFound: ("The function 'marketisVerifierChosenForRound' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')